### Not working well unless pretrained

In [1]:
!pip install efficientnet_pytorch

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1' 

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
import json
from PIL import Image
from torchvision import transforms
from efficientnet_pytorch import EfficientNet

In [5]:
import numpy

In [6]:
from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastprogress import fastprogress
from torchvision.models import *
from fastai.vision.models.xresnet import *
from fastai.vision.models.xresnet2 import *
from fastai.vision.models.presnet import *

In [7]:
torch.backends.cudnn.benchmark = True

# EfficientNet

In [8]:
model_name = 'efficientnet-b1'
image_size = EfficientNet.get_image_size(model_name); image_size 

240

# Data loading

In [9]:
#https://github.com/fastai/fastai/blob/master/examples/train_imagenette.py

def get_data(size, woof, bs, workers=None):
    if   size<=128: path = URLs.IMAGEWOOF_160 if woof else URLs.IMAGENETTE_160
    elif size<=224: path = URLs.IMAGEWOOF_320 if woof else URLs.IMAGENETTE_320
    else          : path = URLs.IMAGEWOOF     if woof else URLs.IMAGENETTE
    path = untar_data(path)

    n_gpus = num_distrib() or 1
    if workers is None: workers = min(8, num_cpus()//n_gpus)

    return (ImageList.from_folder(path).split_by_folder(valid='val')
            .label_from_folder().transform(([flip_lr(p=0.5)], []), size=size)
            .databunch(bs=bs, num_workers=workers)
            .presize(size, scale=(0.35,1))
            .normalize(imagenet_stats))

# Train

In [10]:
opt_func = partial(optim.Adam, betas=(0.9,0.99), eps=1e-6)

In [11]:
# we use the same parameters for baseline and new model
bs = 64
lr = 3e-3
mixup = 0
num_loop = 1

##### New model

In [12]:
def do_cycle(epochs = 5):
    m = EfficientNet.from_name(model_name)
    #m = EfficientNet.from_pretrained(model_name)
    learn = None
    gc.collect()
    learn = (Learner(data, m, wd=1e-2, opt_func=opt_func,
             metrics=[accuracy,top_k_accuracy],
             bn_wd=False, true_wd=True,
             loss_func = LabelSmoothingCrossEntropy())
            )
    if mixup: learn = learn.mixup(alpha=mixup)
    learn = learn.to_fp16(dynamic=True)
    learn.fit_one_cycle(epochs, lr, div_factor=10, pct_start=0.3)
    val_preds,val_targets = learn.get_preds()
    return accuracy(val_preds, tensor(learn.data.valid_ds.y.items)).item()*100

## Imagenette

In [13]:
image_size = 128
data = get_data(image_size,woof =False,bs=bs)

In [14]:
%%time
results = []
for i in range(num_loop):
    results.append(do_cycle(epochs=5))

CPU times: user 13min 53s, sys: 16min 13s, total: 30min 7s
Wall time: 3min 21s


In [15]:
print(results)
print(np.mean(results), np.std(results), np.min(results), np.max(results))

[72.00000286102295]
72.00000286102295 0.0 72.00000286102295 72.00000286102295


In [16]:
image_size = 256
data = get_data(image_size,woof =False,bs=bs)

In [17]:
%%time
results = []
for i in range(num_loop):
    results.append(do_cycle(epochs=5))

CPU times: user 16min 15s, sys: 17min 27s, total: 33min 42s
Wall time: 5min 17s


In [18]:
print(results)
print(np.mean(results), np.std(results), np.min(results), np.max(results))

[74.19999837875366]
74.19999837875366 0.0 74.19999837875366 74.19999837875366


In [19]:
%%time
results = []
for i in range(num_loop):
    results.append(do_cycle(epochs=20))

CPU times: user 1h 5min 23s, sys: 1h 10min 12s, total: 2h 15min 35s
Wall time: 21min 1s


In [20]:
print(results)
print(np.mean(results), np.std(results), np.min(results), np.max(results))

[93.19999814033508]
93.19999814033508 0.0 93.19999814033508 93.19999814033508


## Imagewoof

In [21]:
image_size = 128
data = get_data(image_size,woof =True,bs=bs)

In [22]:
%%time
results = []
for i in range(num_loop):
    results.append(do_cycle(epochs=5))

CPU times: user 13min 42s, sys: 15min 51s, total: 29min 33s
Wall time: 3min 16s


In [23]:
print(results)
print(np.mean(results), np.std(results), np.min(results), np.max(results))

[34.99999940395355]
34.99999940395355 0.0 34.99999940395355 34.99999940395355


In [24]:
image_size = 256
data = get_data(image_size,woof =True,bs=bs)

In [25]:
%%time
results = []
for i in range(num_loop):
    results.append(do_cycle(epochs=5))

CPU times: user 15min 56s, sys: 17min 12s, total: 33min 8s
Wall time: 5min 10s


In [26]:
print(results)
print(np.mean(results), np.std(results), np.min(results), np.max(results))

[42.59999990463257]
42.59999990463257 0.0 42.59999990463257 42.59999990463257


In [27]:
%%time
results = []
for i in range(num_loop):
    results.append(do_cycle(epochs=20))

CPU times: user 1h 2min 58s, sys: 1h 7min 51s, total: 2h 10min 49s
Wall time: 20min 25s


In [28]:
print(results)
print(np.mean(results), np.std(results), np.min(results), np.max(results))

[79.19999957084656]
79.19999957084656 0.0 79.19999957084656 79.19999957084656


In [29]:
%%time
results = []
for i in range(num_loop):
    results.append(do_cycle(epochs=80))

CPU times: user 4h 14min 30s, sys: 4h 33min 33s, total: 8h 48min 3s
Wall time: 1h 21min 58s


In [30]:
print(results)
print(np.mean(results), np.std(results), np.min(results), np.max(results))

[86.19999885559082]
86.19999885559082 0.0 86.19999885559082 86.19999885559082


In [31]:
%%time
results = []
for i in range(num_loop):
    results.append(do_cycle(epochs=400))

CPU times: user 21h 4min 40s, sys: 22h 39min 54s, total: 1d 19h 44min 35s
Wall time: 6h 50min 38s


In [32]:
print(results)
print(np.mean(results), np.std(results), np.min(results), np.max(results))

[89.3999993801117]
89.3999993801117 0.0 89.3999993801117 89.3999993801117
